In [13]:
import pandas as pd
import json
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Separating newer tweets to a test file and older tweets to a train file

In [17]:
df = pd.read_json("data/elon_musk.json")
df = df.sort_values(by="created_at", ascending=False)

In [19]:
test_df = df[:100]
train_df = df[100:]

In [24]:
test = []
train = []

for row in test_df.itertuples():

    temp = {}
    temp["text"] = row.text
    temp["created_at"] = str(row.created_at)
    temp["retweet_count"] = row.retweet_count
    temp["lang"] = row.lang
    temp["prediction"] = row.prediction

    test.append(temp)

for row in train_df.itertuples():

    temp = {}
    temp["text"] = row.text
    temp["created_at"] = str(row.created_at)
    temp["retweet_count"] = row.retweet_count
    temp["lang"] = row.lang
    temp["prediction"] = row.prediction

    train.append(temp)

In [28]:
with open("data/elon_musk_train.json", "w") as f:
    json.dump(train, f)

with open("data/elon_musk_test.json", "w") as f:
    json.dump(test, f)

# Evaluating base model on test dataset

In [5]:
with open("data/elon_musk_test.json", "r") as f:
    data = json.load(f)

In [6]:
def get_prediction(tweet) -> int:
    r = 0
    i = 0
    m = -1

    l = []
    l.append(tweet["prediction"]["LABEL_0"])
    l.append(tweet["prediction"]["LABEL_1"])
    l.append(tweet["prediction"]["LABEL_2"])

    for prob in l:
        if prob > m:
            m = prob
            i = r
        r += 1
    return i

In [7]:
predictions = [get_prediction(t) for t in data]
true_labels = [t["true_label"] for t in data]

In [8]:
acc = 0

for p,t in zip(predictions, true_labels):
    if p == t:
        acc += 1

print(f"Accuracy of base model: {acc/len(predictions)*100:.0f}%")

Accuracy of base model: 57%


In [12]:
f1 = f1_score(true_labels, predictions, average="micro")
precision = precision_score(true_labels, predictions, average="micro")
recall = recall_score(true_labels, predictions, average="micro")

print(f"F1 score: {f1}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

F1 score: 0.57
Precision: 0.57
Recall: 0.57


In [14]:
confusion_matrix(true_labels, predictions)

array([[23,  8,  0],
       [15, 26, 11],
       [ 4,  5,  8]])